In [8]:
#%pip install uproot 
#%pip install awkward 
#%pip install mplhep 
#%pip install numpy 
#%pip install matplotlib 
#%pip install scipy
#%pip install uproot awkward mplhep numpy matplotlib scipy --upgrade

In [9]:
import uproot
import awkward as ak
import mplhep 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.optimize import curve_fit
import scipy.stats as st

In [10]:
var1 = 'Pcharged'
var2 = 'cos_thet'
var3 = 'E_ecal'
var4 = 'Ncharged'
var5 = 'E_hcal'
var6 = 'E_lep'
var7 = 'cos_thru'

### (Un)comment that for the real opal data until "until here"
#path_data = 'opal_data/FOLDERNAME'
#file = uproot.open(path_data+'FILENAME')
#ttree_name = 'myTTree'
#file[ttree_name].keys()
#branches = file[ttree_name].arrays()
#pchar = ak.to_numpy(braches[var1])
#cos_theta = ak.to_numpy(branches[var2])
#Ecal = ak.to_numpy(branches[var3])
#nchar = ak.to_numpy(branches[var4])
#Hcal = ak.to_numpy(branches[var5])
### until here

### This section can later be off commented until it says "until here"
path_data = 'opal_data/mc/'
file1 = uproot.open(path_data+'ee.root')
file2 = uproot.open(path_data+'qq.root')
file3 = uproot.open(path_data+'mm.root')
file4 = uproot.open(path_data+'tt.root')
ttree_name = 'myTTree'
file1[ttree_name].keys()
file2[ttree_name].keys()
file3[ttree_name].keys()
file4[ttree_name].keys()
branches1 = file1[ttree_name].arrays()
branches2 = file2[ttree_name].arrays()
branches3 = file3[ttree_name].arrays()
branches4 = file4[ttree_name].arrays()
## converting the ak arrays into np arrays
##ee
pchar1 = ak.to_numpy(branches1[var1])
cos_theta1 = ak.to_numpy(branches1[var2])
Ecal1 = ak.to_numpy(branches1[var3])
nchar1 = ak.to_numpy(branches1[var4])
Hcal1 = ak.to_numpy(branches1[var5])
##qq
pchar2 = ak.to_numpy(branches2[var1])
cos_theta2 = ak.to_numpy(branches2[var2])
Ecal2 = ak.to_numpy(branches2[var3])
nchar2 = ak.to_numpy(branches2[var4])
Hcal2 = ak.to_numpy(branches2[var5])
##mm
pchar3 = ak.to_numpy(branches3[var1])
cos_theta3 = ak.to_numpy(branches3[var2])
Ecal3 = ak.to_numpy(branches3[var3])
nchar3 = ak.to_numpy(branches3[var4])
Hcal3 = ak.to_numpy(branches3[var5])
##tt
pchar4 = ak.to_numpy(branches4[var1])
cos_theta4 = ak.to_numpy(branches4[var2])
Ecal4 = ak.to_numpy(branches4[var3])
nchar4 = ak.to_numpy(branches4[var4])
Hcal4 = ak.to_numpy(branches4[var5])
### until here

# Applying Electrons cut to Hadrons, Muons and Tau

In [11]:
def eff(x):
    return sum(x)/100000

In [12]:
### These lines calculate the diagonal efficiencies --- 'mw' means mynewmask as usually
mw11 = (branches1['E_ecal'] <= 120) & (branches1['E_ecal'] >= 70) & (branches1['Ncharged'] <= 5) & (branches1['cos_thet'] >= -0.5) & (branches1['cos_thet'] <= 0.5)
############################################ Seperation of s- and t-channel
# Calculating the theoretical events in range [-1,0.5] and [0.5,1] by integration of do/dthet = A(1+x^2) in the range of [-1,1] and [-0.5,0.5] and then taking the ratio of it... 
ratio = (25*3)/(24*4) ### this is the amount of data in the range of [-0.5,0.5] considering all datapoints, thus 
ratio_ = 1/ratio ### is the amount of data in the othe regions;)
N_total =  sum(mw11)*ratio_
N_cut = sum(mw11)
N_cutted = N_total - N_cut
########################################################
mw22 = (branches2['Ncharged'] >= 10) & (branches2['Ncharged'] <= 60) 
mw33 = (branches3['Ncharged'] <= 3) & (branches3['E_ecal'] <= 10) 
mw44 = (branches4['Ncharged'] <= 8) & (branches4['E_ecal'] >= 10) & (branches4['E_ecal'] <= 60)
eff11 = sum(mw11)/N_total
eff22 = eff(mw22)
eff33 = eff(mw33)
eff44 = eff(mw44)
print(eff11)

0.78125


In [13]:
### These lines calculate the diagonal efficiencies---- 'mw' means mynewmask as usually
## '12' means electron row and hadron column 
mw12 = (branches2['E_ecal'] <= 120) & (branches2['E_ecal'] >= 70) & (branches2['Ncharged'] <= 5) & (branches2['cos_thet'] >= -0.5) & (branches2['cos_thet'] <= 0.5)
mw13 = (branches3['E_ecal'] <= 120) & (branches3['E_ecal'] >= 70) & (branches3['Ncharged'] <= 5) & (branches3['cos_thet'] >= -0.5) & (branches3['cos_thet'] <= 0.5)
mw14 = (branches4['E_ecal'] <= 120) & (branches4['E_ecal'] >= 70) & (branches4['Ncharged'] <= 5) & (branches4['cos_thet'] >= -0.5) & (branches4['cos_thet'] <= 0.5)
## '21' means hadron rwo and electron column
mw21 = (branches1['Ncharged'] >= 10) & (branches1['Ncharged'] <= 60) 
mw23 = (branches3['Ncharged'] >= 10) & (branches3['Ncharged'] <= 60) 
mw24 = (branches4['Ncharged'] >= 10) & (branches4['Ncharged'] <= 60) 
##
mw31 = (branches1['Ncharged'] <= 3) & (branches1['E_ecal'] <= 10) 
mw32 = (branches2['Ncharged'] <= 3) & (branches2['E_ecal'] <= 10) 
mw34 = (branches4['Ncharged'] <= 3) & (branches4['E_ecal'] <= 10) 
##
mw41 = (branches1['Ncharged'] <= 8) & (branches1['E_ecal'] >= 10) & (branches1['E_ecal'] <= 60)
mw42 = (branches2['Ncharged'] <= 8) & (branches2['E_ecal'] >= 10) & (branches2['E_ecal'] <= 60)
mw43 = (branches3['Ncharged'] <= 8) & (branches3['E_ecal'] >= 10) & (branches3['E_ecal'] <= 60)
###
eff12 = eff(mw12)
eff13 = eff(mw13)
eff14 = eff(mw14)
#
eff21 = eff(mw21)
eff23 = eff(mw23)
eff24 = eff(mw24)
#
eff31 = eff(mw31)
eff32 = eff(mw32)
eff34 = eff(mw34)
#
eff41 = eff(mw41)
eff42 = eff(mw42)
eff43 = eff(mw43)

## creating a data matrix
trydata = np.asarray([sum(mw11),sum(mw22),sum(mw33),sum(mw44)])

In [14]:
## define efficiency matrix
eff_all = np.asarray([[eff11,eff12,eff13,eff14],[eff21,eff22,eff23,eff24],[eff31,eff32,eff33,eff34],[eff41,eff42,eff43,eff44]])
np.save('eff_matrix',eff_all)
print(eff_all)

[[0.78125 0.      0.      0.00139]
 [0.      0.95307 0.      0.00148]
 [0.      0.      0.87387 0.07373]
 [0.00532 0.01449 0.06399 0.67306]]


# doing the backward check

In [19]:
## also backward check 
inv_eff_all = np.linalg.inv(eff_all)
check = inv_eff_all*eff_all
print(inv_eff_all)
print(check)

[[ 1.28001815e+00  4.05166109e-05  1.95143821e-04 -2.66495282e-03]
 [ 1.58388364e-05  1.04927624e+00  1.70320511e-04 -2.32595694e-03]
 [ 8.60565161e-04  1.92134672e-03  1.15358893e+00 -1.26375288e-01]
 [-1.01996755e-02 -2.27723757e-02 -1.09680655e-01  1.49783769e+00]]
[[ 1.00001418e+00  0.00000000e+00  0.00000000e+00 -3.70428442e-06]
 [ 0.00000000e+00  1.00003370e+00  0.00000000e+00 -3.44241627e-06]
 [ 0.00000000e+00  0.00000000e+00  1.00808675e+00 -9.31764998e-03]
 [-5.42622738e-05 -3.29971724e-04 -7.01846510e-03  1.00813464e+00]]


In [16]:
testrun = inv_eff_all.dot(trydata)
print(testrun)
print(type(testrun))

[17437.95670581 99861.92098876 92497.80832032 88918.21840665]
<class 'numpy.ndarray'>
